In [3]:

from NDPopulation import NDPopulation
from ProcessResult import ProcessResult

In [1]:
import glob
from ElectionConstructor import *
import pickle
from NDPopulation import NDPopulation

def load_path(path: str) -> any:
    with open(path, "rb") as f:
        return pickle.load(f)



def generate_results(candidates: List[Candidate], processes: List[ElectionConstructor]) -> List[ElectionResult]:
    dim = candidates[0].ideology.dim
    population = NDPopulation(np.zeros((dim,)), np.ones((dim,)))
    voters = population.generate_unit_voters(1000)
    ballots = [Ballot(v, candidates, unit_election_config) for v in voters]
    return [p.run(ballots, set(candidates)).winner() for p in processes]

def compare_outcomes():
    irv_results: List[ProcessResult] = [load_path(s) for s in glob.glob("process_results-2/Hare*.p")]

    irv_results.sort(key=lambda r: r.dim)
    processes = [
        ElectionConstructor(constructor=construct_irv, name="Hare"),
        ElectionConstructor(constructor=construct_h2h, name="Minimax")
    ]

    print("dimensionality  agree disagree %agree")
    for ir in irv_results:
        dim = ir.dim
        agree_count = 0
        disagree_count = 0
        for w, cc in ir.stats.results[0:1000]:
            o = generate_results(cc, processes)
            if o[0].name == o[1].name:
                agree_count += 1
            else:
                disagree_count += 1

        print("%d              %4d  %4d      %5.2f%%" % (dim, agree_count, disagree_count, 100 * agree_count / (agree_count + disagree_count)))

compare_outcomes()

dimensionality  agree disagree %agree
1               573   427      57.30%
2               808   192      80.80%
3               905    95      90.50%
4               932    68      93.20%


In [2]:
def print_winner_stats():
    irv_results: List[ProcessResult] = [load_path(s) for s in glob.glob("process_results-2/Hare*.p")]
    irv_results.sort(key=lambda r: r.dim)

    h2h_results: List[ProcessResult] = [load_path(s) for s in glob.glob("process_results-2/Minimax*.p")]
    h2h_results.sort(key=lambda r: r.dim)

    def get_winner_distance(result: ProcessResult) -> np.array:
        v = [w.ideology.distance_from_o() for w, cc in result.stats.results]
        return np.array(v)

    print("Dimensionality  Hare      Minimax")
    dim = 1
    for ir, hr in zip(irv_results, h2h_results):
        ird = get_winner_distance(ir)
        hd = get_winner_distance(hr)
        i_mean: float = np.mean(ird)
        h_mean: float = np.mean(hd)
        print("%d              %5.2f     %5.2f" % (dim, i_mean, h_mean))
        dim += 1

print_winner_stats()

NameError: name 'ProcessResult' is not defined